In [35]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
# Load the TensorBoard notebook extension
%reload_ext tensorboard

2.1.0


In [36]:
!rm -rf ./logs/ 

In [37]:
fashion_mnist = keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [38]:
#OPTIONAL
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [3]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [11]:
#we need to grainscale images

In [15]:
#building model

In [50]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ])

    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(x_train, y_train, epochs=2)
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [51]:
#For each run, log an hparams summary with the hyperparameters and final accuracy:

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

Source until now: https://www.tensorflow.org/tutorials/keras/classification

In [52]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
              HP_NUM_UNITS: num_units,
              HP_DROPOUT: dropout_rate,
              HP_OPTIMIZER: optimizer,
          }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('./logs/hparam_tuning/' + run_name, hparams)
            session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 68us/sample - loss: 0.7082 - accuracy: 0.7562
Epoch 2/2
10000/10000 [==============================] - 0s 43us/sample - loss: 0.4538 - accuracy: 0.8396
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 62us/sample - loss: 0.9955 - accuracy: 0.6628
Epoch 2/2
10000/10000 [==============================] - 0s 41us/sample - loss: 0.5675 - accuracy: 0.8080
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 72us/sample - loss: 0.7935 - accuracy: 0.7258
Epoch 2/2
10000/10000 [==============================] - 1s 52us/sample - loss: 0.4836 - accuracy: 0.8246
--- Starting trial: run-3
{'num_units': 16, 'dropout'

In [53]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 1554), started 0:22:27 ago. (Use '!kill 1554' to kill it.)